In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import math

Adapting datasets to be interpreted

In [2]:
X_sample = pd.read_csv("data/ais_train.csv", delimiter='|')
X_sample.to_csv('data/ais_train_modified.csv', index=False)
extra_vessels = pd.read_csv("data/vessels.csv", on_bad_lines='skip', delimiter='|')
extra_vessels.to_csv('data/vessels_modified.csv', index=False)
extra_ports = pd.read_csv("data/ports.csv", on_bad_lines='skip', delimiter='|')
extra_ports.to_csv('data/ports_modified.csv', index=False)
extra_schedules = pd.read_csv("data/schedules_to_may_2024.csv", on_bad_lines='skip', delimiter='|')
extra_schedules.to_csv('data/schedules_to_may_2024_modified.csv', index=False)

In [3]:
X_evaluation = pd.read_csv("data/ais_test.csv",)
extra_ports = pd.read_csv("data/ports_modified.csv")
extra_vessels = pd.read_csv("data/vessels_modified.csv")
extra_schedules = pd.read_csv("data/schedules_to_may_2024_modified.csv")
X_original = pd.read_csv("data/ais_train_modified.csv")

Changing current data into previous data for the train file

In [4]:
def past_course(original):
    original=original.reset_index()

    original['prev_lat'] = original['latitude'].shift(1).fillna(original['latitude'].iloc[0])
    original['prev_lon'] = original['longitude'].shift(1).fillna(original['longitude'].iloc[0])
    original['time_2'] = original['time'].shift(1).fillna(original['time'].iloc[0])
    original['cog'] = original['cog'].shift(1)
    original['sog'] = original['sog'].shift(1)
    original['rot'] = original['rot'].shift(1)
    original['heading'] = original['heading'].shift(1)
    original['navstat'] = original['navstat'].shift(1)
    original.loc[0,['cog','sog','rot','heading','navstat']]=[0,0,0,0,0]

    return original

In [5]:
def adapting_training_data(original):
    unique = original['vesselId'].unique()
    new = original.copy()
    new['time'] = pd.to_datetime(new['time'])
    new['prev_lat']=original['latitude']
    new['prev_lon']=original['longitude']
    new['time_2'] = new['time']
    new = new.reset_index()
    final = pd.DataFrame(columns=new.columns)
    for c in unique:
        filtered = new[new['vesselId'] == c].copy()
        new_filtered = past_course(filtered)
        final = pd.concat([final, new_filtered], ignore_index=True)
    final = final.sort_values(by='index')
    final = final.drop(['index'],axis=1)
    final = final.reset_index(drop=True)
    final = final.drop(['level_0'],axis=1)
    return(final)

In [6]:
testeo = new_dataframe(X_original)

C:\Users\jtlas\AppData\Local\Temp\ipykernel_7140\561889474.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final, new_filtered], ignore_index=True)


In [44]:
X_original.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [ ]:
x = testeo[testeo['vesselId'] == '61e9f3a8b937134a3c4bfdf7'].copy()
x.head()

In [58]:
def travel_predict(lon,lat,speed, time, direction):
    speed = speed *0.514444*3600
    dist = (speed * time)/1000
    lat = lat * (math.pi/180)
    lon = lon * (math.pi/180)
    direction = direction * (math.pi/180)
    R=6371
    latf = math.asin(math.sin(lat)*math.cos(dist/R) + math.cos(lat)*math.sin(dist/R)*math.cos(direction))
    lonf = lon + math.atan((math.sin(direction)*math.sin(dist/R)*math.cos(lat))/(math.cos(dist/R)-(math.sin(lat)*math.sin(latf))))
    latf = latf*(180/math.pi)
    lonf = lonf*(180/math.pi)
    return (lonf,latf)
def travel_predict_1(df):
    lat = df['prev_lat'] * (np.pi / 180)
    lon = df['prev_lon'] * (np.pi / 180)
    speed = df['sog'] * 0.514444 * 3600
    dist = (speed * df['time_dif']) / 1000
    direction = df['cog'] * (np.pi / 180)
    R = 6371

    latf = np.arcsin(np.sin(lat) * np.cos(dist / R) + np.cos(lat) * np.sin(dist / R) * np.cos(direction))
    lonf = lon + np.arctan((np.sin(direction) * np.sin(dist / R) * np.cos(lat)) /(np.cos(dist / R) - (np.sin(lat) * np.sin(latf))))

    latf = latf * (180 / np.pi)
    lonf = lonf * (180 / np.pi)

    result = pd.DataFrame({'longitude_predicted': lonf, 'latitude_predicted': latf})

    return result

def adapting_test_data (evaluation,training):
    evalu=evaluation.copy()
    evalu['time'] = pd.to_datetime(evalu['time'])
    evalu['time_2'] = evalu['time']
    evalu['cog'] = 0.1
    evalu['sog'] = 0.1
    evalu['heading'] = 0.1
    evalu['navstat'] = 0.1
    evalu['etaRaw'] = evalu['time']
    evalu['latitude'] = 0.11111111
    evalu['longitude'] = 0.11111111
    evalu['portId'] ='61d371c43aeaecc07011a37f'
    train=training.copy()
    train['time'] = pd.to_datetime(train['time'])
    train['time_2'] = train['time']
    evalu = evalu.drop(['ID','scaling_factor'],axis=1)
    final = pd.concat([train, evalu], ignore_index=True)
    x =new_dataframe(final)
    evalu = x.iloc[len(train):]
    unique = evalu['vesselId'].unique()
    final = final.iloc[0:0]
    for c in unique:
        filtered = evalu[evalu['vesselId'] == c].copy()
        filtered = filtered.reset_index()
        filtered[['time_2','cog','sog','rot','heading','navstat','etaRaw','prev_lat','prev_lon']] = filtered.iloc[0][['time_2','cog','sog','rot','heading','navstat','etaRaw','prev_lat','prev_lon']]
        final = pd.concat([final,filtered], ignore_index=True)
    final['time_dif'] = (final['time']-final['time_2']).dt.total_seconds()/3600
    final = final.sort_values(by='index')
    final = final.drop(['index'],axis=1)
    final = final.reset_index(drop=True)
    #final = final.drop(['level_0'],axis=1)
    return final

In [ ]:
 #   pred = evaluation.copy()
  #  pred = pred.drop(['vesselId','time','scaling_factor'],axis=1)
   # pred['longitude_predicted']=0
   # pred['latitude_predicted']=0
t = assignation(X_evaluation,X_original)

In [39]:
x = t.copy()
x = x.drop(['latitude','longitude','time_2','prev_lat','prev_lon','time_dif','vesselId','portId'],axis=1)
x['time'] = pd.to_datetime(x['time'])
x['year'] = x['time'].dt.year
x['month'] = x['time'].dt.month
x['day'] = x['time'].dt.day
x['hour'] = x['time'].dt.hour
x['minute'] = x['time'].dt.minute
x['second'] = x['time'].dt.second

x['etaRaw'] = pd.to_datetime(x['etaRaw'], format='%m-%d %H:%M', errors='coerce')
x['eta_month'] = x['etaRaw'].dt.month
x['eta_day'] = x['etaRaw'].dt.day
x['eta_hour'] = x['etaRaw'].dt.hour
x['eta_minute'] = x['etaRaw'].dt.minute

x = x.drop(['time','etaRaw'], axis=1)


In [48]:
t.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,time_2,prev_lat,prev_lon,time_dif
0,2024-05-08 00:03:16,179.6,0.0,0.0,344.0,5.0,2024-05-08 00:03:16,0.111111,0.111111,61e9f3aeb937134a3c4bfe3d,61d371c43aeaecc07011a37f,2024-05-07 23:48:16,31.14647,-81.49789,0.250000
1,2024-05-08 00:06:17,24.7,0.0,0.0,214.0,5.0,2024-05-08 00:06:17,0.111111,0.111111,61e9f473b937134a3c4c02df,61d371c43aeaecc07011a37f,2024-05-07 23:57:16,14.81694,120.29625,0.150278
2,2024-05-08 00:10:02,8.0,18.7,0.0,6.0,0.0,2024-05-08 00:10:02,0.111111,0.111111,61e9f469b937134a3c4c029b,61d371c43aeaecc07011a37f,2024-05-07 23:59:08,38.27895,10.78280,0.181667
3,2024-05-08 00:10:34,321.3,0.1,0.0,70.0,1.0,2024-05-08 00:10:34,0.111111,0.111111,61e9f45bb937134a3c4c0221,61d371c43aeaecc07011a37f,2024-05-07 23:52:34,-43.53785,172.83522,0.300000
4,2024-05-08 00:12:27,291.0,0.3,0.0,275.0,2.0,2024-05-08 00:12:27,0.111111,0.111111,61e9f38eb937134a3c4bfd8d,61d371c43aeaecc07011a37f,2024-05-07 23:51:29,48.53320,-6.12003,0.349444


In [47]:
def prediction (test):
    results = []
    for i in range(len(test)):
        pos = travel_predict(test.loc[i,['prev_lon']],test.loc[i,['prev_lat']],test.loc[i,['sog']],test.loc[i,['time_dif']],test.loc[i,['cog']])
        results.append({'ID':i,'longitude_predicted':pos[0],'latitude_predicted':pos[1]})
    pred = pd.Dataframe(results)
        
        

In [65]:
#pos = travel_predict(test.loc[i,['prev_lon']],test.loc[i,['prev_lat']],test.loc[i,['sog']],test.loc[i,['time_dif']],test.loc[i,['cog']])
result =travel_predict_1(t)
result=result.reset_index()
result.rename(columns={'index': 'ID'}, inplace=True)
result.to_csv('data/first_try.csv', index=False)

Changing the dates into a different columns

In [13]:
X_original['time'] = pd.to_datetime(X_original['time'])
X_original['year'] = X_original['time'].dt.year
X_original['month'] = X_original['time'].dt.month
X_original['day'] = X_original['time'].dt.day
X_original['hour'] = X_original['time'].dt.hour
X_original['minute'] = X_original['time'].dt.minute
X_original['second'] = X_original['time'].dt.second

X_original['etaRaw'] = pd.to_datetime(X_original['etaRaw'], format='%m-%d %H:%M', errors='coerce')
X_original['eta_month'] = X_original['etaRaw'].dt.month
X_original['eta_day'] = X_original['etaRaw'].dt.day
X_original['eta_hour'] = X_original['etaRaw'].dt.hour
X_original['eta_minute'] = X_original['etaRaw'].dt.minute

X_original = X_original.drop(['time','etaRaw'], axis=1)


Code to generate a merged dataset with the vessels (non functional)

In [ ]:
prueba = pd.merge(X_train,extra_vessels, on='vesselId',how='left')
prueba.head()

Splitting the dataset into a train/test one and x-(all the info)  y-(the results longitud/latitud)

In [14]:
#Generating one test file
X_original = X_original.drop(['vesselId','portId'], axis=1)

x_test=X_original.sample(frac=0.2, random_state=42)
x_train=X_original.drop(x_test.index)

y_test_lon=x_test.loc[:,['longitude']]
y_test_lat=x_test.loc[:,['latitude']]
x_test=x_test.drop(['longitude','latitude'],axis=1)

y_train_lon=x_train.loc[:,['longitude']]
y_train_lat=x_train.loc[:,['latitude']]
x_train = x_train.drop(['longitude','latitude'],axis=1)

In [ ]:
#verification (length must match)
print(f"x_test shape: {x_test.shape}")
print(f"y_test_lon shape: {y_test_lon.shape}")
print(f"y_test_lat shape: {y_test_lat.shape}")

In [15]:
#Generating three sets of training data to train 3 different models
#Split of the data in 3 equal slices
x_original_1 = X_original.sample(frac=1/3, random_state=42)
X_original_trans = X_original.drop(x_original_1.index)
x_original_2 = X_original_trans.sample(frac=0.5, random_state=42)
x_original_3 = X_original_trans.drop(x_original_2.index)

#Adapting the slices into x and y
#slice 1
y_test_1_lon = x_original_1.loc[:,['longitude']]
y_test_1_lat = x_original_1.loc[:,['latitude']]
x_test_1 = x_original_1.drop(['longitude','latitude'],axis=1)
x_train_1 = X_original.drop(x_original_1.index)
y_train_1_lon = x_train_1.loc[:,['longitude']]
y_train_1_lat = x_train_1.loc[:,['latitude']]
x_train_1 = x_train_1.drop(['longitude','latitude'],axis=1)
#slice 2
y_test_2_lon=x_original_2.loc[:,['longitude']]
y_test_2_lat=x_original_2.loc[:,['latitude']]
x_test_2 = x_original_2.drop(['longitude','latitude'],axis=1)
x_train_2 = X_original.drop(x_original_2.index)
y_train_2_lon = x_train_2.loc[:,['longitude']]
y_train_2_lat = x_train_2.loc[:,['latitude']]
x_train_2 = x_train_2.drop(['longitude','latitude'],axis=1)
#slice 3
y_test_3_lon=x_original_3.loc[:,['longitude']]
y_test_3_lat=x_original_3.loc[:,['latitude']]
x_test_3 = x_original_3.drop(['longitude','latitude'],axis=1)
x_train_3 = X_original.drop(x_original_3.index)
y_train_3_lon = x_train_3.loc[:,['longitude']]
y_train_3_lat = x_train_3.loc[:,['latitude']]
x_train_3 = x_train_3.drop(['longitude','latitude'],axis=1)

In [18]:
#verification (length must match)
print(f"X_original shape: {X_original.shape}")
print(f"x_train_1 shape: {x_train_1.shape}")
print(f"y_train_1_lon shape: {y_train_1_lon.shape}")
print(f"y_train_1_lat shape: {y_train_1_lat.shape}")
print(f"x_test_1 shape: {x_test_1.shape}")
print(f"y_test_1_lon shape: {y_test_1_lon.shape}")
print(f"y_test_1_lat shape: {y_test_1_lat.shape}")



X_original shape: (1522065, 17)
x_train_1 shape: (1014710, 15)
y_train_1_lon shape: (1014710, 1)
y_train_1_lat shape: (1014710, 1)
x_test_1 shape: (507355, 15)
y_test_1_lon shape: (507355, 1)
y_test_1_lat shape: (507355, 1)


Function to evaluate results

In [41]:
# define a utility function to print out the prediction performance
def evaluate_result(y_test, y_pred):
    print(mean_absolute_error(y_test, y_pred))
    print(r2_score(y_test, y_pred))

Model generation of random forest

In [ ]:
clf = RandomForestRegressor(n_estimators=100, random_state=42)
clf.fit(x_train, y_train_lon.values.ravel())
y_pred = clf.predict(x_test)

In [68]:
evaluate_result(y_test_lon,y_pred,clf)

15.217318774346422
0.7901448046453818


Hyperparameter tuning for random forest regressor

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# model definition
rf_regressor = RandomForestRegressor()

# Search settings
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# adjust the model
grid_search.fit(x_train, y_train_lon)

# see best parameters
print(grid_search.best_params_)

Function to build the submission file

In [1]:
def result_merger(longitude,latitude):
    submission = pd.merge(longitude, latitude, left_index=True, right_index=True, how='inner')
    submission.insert(0, 'ID', range(len(df)))
    submission.to_csv('submission.csv', index=False)
    return submission